#Задание 1:
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

**Задача**:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU

In [ ]:
import torch
from torch import nn
import random
import tqdm
import time

In [ ]:
tensor_10 = torch.randint(10, (1000,10))
tensor_10[0:3]

tensor([[7, 7, 9, 4, 0, 2, 9, 8, 0, 4],
        [5, 8, 3, 7, 0, 8, 4, 3, 5, 7],
        [0, 9, 7, 3, 6, 2, 0, 6, 2, 8]])

In [ ]:
tensor_40 = torch.randint(10, (1000,40))
tensor_40[0:3]

tensor([[4, 1, 0, 0, 8, 1, 2, 3, 0, 0, 8, 5, 8, 4, 3, 3, 9, 8, 9, 8, 6, 9, 9, 2,
         5, 1, 7, 5, 7, 8, 4, 1, 6, 0, 5, 9, 9, 8, 1, 6],
        [8, 6, 9, 7, 1, 7, 7, 6, 9, 0, 2, 8, 4, 2, 6, 5, 9, 1, 4, 1, 4, 2, 1, 6,
         0, 3, 6, 9, 3, 4, 6, 1, 5, 2, 7, 6, 8, 3, 8, 2],
        [8, 6, 9, 1, 5, 0, 3, 1, 0, 9, 0, 4, 8, 2, 2, 5, 8, 8, 7, 5, 0, 0, 8, 9,
         7, 0, 8, 3, 1, 0, 9, 0, 0, 9, 2, 5, 1, 0, 0, 7]])

In [ ]:
tensor_80 = torch.randint(10, (1000,80))
tensor_80[0:3]

tensor([[0, 4, 5, 6, 2, 8, 4, 3, 1, 0, 8, 8, 0, 3, 3, 2, 4, 7, 0, 7, 2, 0, 9, 5,
         1, 2, 0, 0, 3, 9, 6, 2, 6, 9, 8, 5, 1, 8, 2, 4, 7, 5, 1, 0, 6, 2, 3, 2,
         3, 4, 9, 1, 5, 2, 0, 7, 6, 8, 3, 7, 3, 4, 1, 4, 5, 0, 4, 7, 1, 0, 4, 5,
         4, 7, 9, 0, 5, 8, 2, 0],
        [3, 3, 4, 7, 7, 7, 2, 0, 4, 4, 1, 1, 6, 6, 9, 5, 6, 3, 2, 7, 7, 8, 7, 0,
         9, 6, 2, 9, 1, 2, 3, 9, 0, 5, 4, 8, 3, 4, 7, 7, 1, 7, 1, 5, 1, 2, 6, 5,
         3, 2, 0, 3, 6, 6, 0, 2, 6, 6, 9, 1, 8, 5, 5, 3, 9, 5, 5, 7, 3, 2, 9, 4,
         1, 5, 7, 4, 3, 3, 6, 4],
        [1, 0, 3, 0, 2, 3, 9, 2, 2, 5, 0, 5, 5, 4, 2, 0, 1, 2, 5, 1, 0, 2, 4, 1,
         9, 6, 2, 4, 0, 1, 6, 4, 3, 5, 1, 6, 7, 6, 9, 6, 5, 4, 9, 3, 2, 1, 5, 0,
         1, 2, 0, 3, 1, 9, 7, 3, 5, 1, 1, 4, 5, 9, 6, 5, 3, 6, 0, 1, 2, 4, 1, 0,
         4, 3, 9, 5, 6, 1, 4, 6]])

In [ ]:
def transform(tensor):
    y = torch.zeros_like(tensor)
    for j in range(len(y)):
        for i in range(len(y[1,:])):
            y[j,0] = tensor[j,0]
            #y[j,i] = (y[j,i-1]+tensor[j,i]) % 10
            y[j,i] = (tensor[j,i]+y[j,0]) % 10
    return y

In [ ]:
transform_tensor_10 = transform(tensor_10)
tensor_10[:3], transform_tensor_10[:3]

(tensor([[7, 7, 9, 4, 0, 2, 9, 8, 0, 4],
         [5, 8, 3, 7, 0, 8, 4, 3, 5, 7],
         [0, 9, 7, 3, 6, 2, 0, 6, 2, 8]]),
 tensor([[7, 4, 6, 1, 7, 9, 6, 5, 7, 1],
         [5, 3, 8, 2, 5, 3, 9, 8, 0, 2],
         [0, 9, 7, 3, 6, 2, 0, 6, 2, 8]]))

In [ ]:
transform_tensor_40 = transform(tensor_40)
tensor_40[:3], transform_tensor_40[:3]

(tensor([[4, 1, 0, 0, 8, 1, 2, 3, 0, 0, 8, 5, 8, 4, 3, 3, 9, 8, 9, 8, 6, 9, 9, 2,
          5, 1, 7, 5, 7, 8, 4, 1, 6, 0, 5, 9, 9, 8, 1, 6],
         [8, 6, 9, 7, 1, 7, 7, 6, 9, 0, 2, 8, 4, 2, 6, 5, 9, 1, 4, 1, 4, 2, 1, 6,
          0, 3, 6, 9, 3, 4, 6, 1, 5, 2, 7, 6, 8, 3, 8, 2],
         [8, 6, 9, 1, 5, 0, 3, 1, 0, 9, 0, 4, 8, 2, 2, 5, 8, 8, 7, 5, 0, 0, 8, 9,
          7, 0, 8, 3, 1, 0, 9, 0, 0, 9, 2, 5, 1, 0, 0, 7]]),
 tensor([[4, 5, 4, 4, 2, 5, 6, 7, 4, 4, 2, 9, 2, 8, 7, 7, 3, 2, 3, 2, 0, 3, 3, 6,
          9, 5, 1, 9, 1, 2, 8, 5, 0, 4, 9, 3, 3, 2, 5, 0],
         [8, 4, 7, 5, 9, 5, 5, 4, 7, 8, 0, 6, 2, 0, 4, 3, 7, 9, 2, 9, 2, 0, 9, 4,
          8, 1, 4, 7, 1, 2, 4, 9, 3, 0, 5, 4, 6, 1, 6, 0],
         [8, 4, 7, 9, 3, 8, 1, 9, 8, 7, 8, 2, 6, 0, 0, 3, 6, 6, 5, 3, 8, 8, 6, 7,
          5, 8, 6, 1, 9, 8, 7, 8, 8, 7, 0, 3, 9, 8, 8, 5]]))

In [ ]:
transform_tensor_80 = transform(tensor_80)
tensor_80[:3], transform_tensor_80[:3]

(tensor([[0, 4, 5, 6, 2, 8, 4, 3, 1, 0, 8, 8, 0, 3, 3, 2, 4, 7, 0, 7, 2, 0, 9, 5,
          1, 2, 0, 0, 3, 9, 6, 2, 6, 9, 8, 5, 1, 8, 2, 4, 7, 5, 1, 0, 6, 2, 3, 2,
          3, 4, 9, 1, 5, 2, 0, 7, 6, 8, 3, 7, 3, 4, 1, 4, 5, 0, 4, 7, 1, 0, 4, 5,
          4, 7, 9, 0, 5, 8, 2, 0],
         [3, 3, 4, 7, 7, 7, 2, 0, 4, 4, 1, 1, 6, 6, 9, 5, 6, 3, 2, 7, 7, 8, 7, 0,
          9, 6, 2, 9, 1, 2, 3, 9, 0, 5, 4, 8, 3, 4, 7, 7, 1, 7, 1, 5, 1, 2, 6, 5,
          3, 2, 0, 3, 6, 6, 0, 2, 6, 6, 9, 1, 8, 5, 5, 3, 9, 5, 5, 7, 3, 2, 9, 4,
          1, 5, 7, 4, 3, 3, 6, 4],
         [1, 0, 3, 0, 2, 3, 9, 2, 2, 5, 0, 5, 5, 4, 2, 0, 1, 2, 5, 1, 0, 2, 4, 1,
          9, 6, 2, 4, 0, 1, 6, 4, 3, 5, 1, 6, 7, 6, 9, 6, 5, 4, 9, 3, 2, 1, 5, 0,
          1, 2, 0, 3, 1, 9, 7, 3, 5, 1, 1, 4, 5, 9, 6, 5, 3, 6, 0, 1, 2, 4, 1, 0,
          4, 3, 9, 5, 6, 1, 4, 6]]),
 tensor([[0, 4, 5, 6, 2, 8, 4, 3, 1, 0, 8, 8, 0, 3, 3, 2, 4, 7, 0, 7, 2, 0, 9, 5,
          1, 2, 0, 0, 3, 9, 6, 2, 6, 9, 8, 5, 1, 8, 2, 4, 7, 5, 1, 0, 6, 

In [ ]:
BATCH_SIZE = 128
dataset_10 = torch.utils.data.TensorDataset(tensor_10, transform_tensor_10)
data_10 = torch.utils.data.DataLoader(dataset_10, BATCH_SIZE, shuffle=True)

dataset_40 = torch.utils.data.TensorDataset(tensor_40, transform_tensor_40)
data_40 = torch.utils.data.DataLoader(dataset_40, BATCH_SIZE, shuffle=True)

dataset_80 = torch.utils.data.TensorDataset(tensor_80, transform_tensor_80)
data_80 = torch.utils.data.DataLoader(dataset_80, BATCH_SIZE, shuffle=True)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        #_, state = self.hidden(out)
        #predictions = self.output(state[0])
        out, state = self.hidden(out)
        predictions = self.output(out)
        if isinstance(self.hidden, nn.LSTM):
            predictions = predictions.squeeze(0)
        return predictions

In [ ]:
def train_model(model, loader, n_epochs):
    train_losses = []
    #train_accuracy = []
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

    for epoch in range(1, n_epochs + 1):
        start = time.time()
        train_loss, train_acc, n = 0.0, 0.0, 0
        
        model.train()
        for X, y in loader:
            optimizer.zero_grad()
            y_pred = model.forward(X)
            loss = criterion(y_pred, y)#.float())
            train_loss += loss.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            n += y.shape[0]*y.shape[1]
            loss.backward()
            optimizer.step()

        train_loss /= len(loader)
        train_losses.append(train_loss)
        sec = time.time() - start
        stat = f'Epoch: {epoch}, time: {sec:.1f}s, loss: {train_loss:.3f}, acc:{train_acc/n:.3f} '
        if (epoch) % 200 == 0:
            print(stat)

        
    return model, train_losses

In [ ]:
embedding_size = 64
num_hiddens = 128
n_epochs = 5000

In [ ]:
model_RNN_10 = NeuralNetwork(nn.RNN, 10 , embedding_size, num_hiddens, 10)
model, train_losses = train_model(model_RNN_10, data_10, n_epochs)

Epoch: 200, time: 0.1s, loss: 2.085, acc:0.221 
Epoch: 400, time: 0.1s, loss: 1.962, acc:0.303 
Epoch: 600, time: 0.1s, loss: 1.762, acc:0.389 
Epoch: 800, time: 0.1s, loss: 1.542, acc:0.471 
Epoch: 1000, time: 0.1s, loss: 1.325, acc:0.561 
Epoch: 1200, time: 0.1s, loss: 1.120, acc:0.640 
Epoch: 1400, time: 0.1s, loss: 0.937, acc:0.709 
Epoch: 1600, time: 0.1s, loss: 0.785, acc:0.768 
Epoch: 1800, time: 0.1s, loss: 0.638, acc:0.821 
Epoch: 2000, time: 0.1s, loss: 0.532, acc:0.859 
Epoch: 2200, time: 0.1s, loss: 0.512, acc:0.855 
Epoch: 2400, time: 0.1s, loss: 0.414, acc:0.889 
Epoch: 2600, time: 0.1s, loss: 0.343, acc:0.911 
Epoch: 2800, time: 0.1s, loss: 0.698, acc:0.766 
Epoch: 3000, time: 0.1s, loss: 0.276, acc:0.923 
Epoch: 3200, time: 0.1s, loss: 0.259, acc:0.927 
Epoch: 3400, time: 0.1s, loss: 0.233, acc:0.932 
Epoch: 3600, time: 0.1s, loss: 0.220, acc:0.932 
Epoch: 3800, time: 0.1s, loss: 0.205, acc:0.936 
Epoch: 4000, time: 0.1s, loss: 0.195, acc:0.938 
Epoch: 4200, time: 0.1s,

In [ ]:
model_RNN_40 = NeuralNetwork(nn.RNN, 40, embedding_size, num_hiddens, 40)
model, train_losses = train_model(model_RNN_40, data_40, n_epochs)

Epoch: 200, time: 0.2s, loss: 2.323, acc:0.126 
Epoch: 400, time: 0.2s, loss: 2.280, acc:0.135 
Epoch: 600, time: 0.2s, loss: 2.260, acc:0.146 
Epoch: 800, time: 0.2s, loss: 2.245, acc:0.156 
Epoch: 1000, time: 0.2s, loss: 2.231, acc:0.167 
Epoch: 1200, time: 0.3s, loss: 2.218, acc:0.175 
Epoch: 1400, time: 0.3s, loss: 2.206, acc:0.183 
Epoch: 1600, time: 0.3s, loss: 2.193, acc:0.191 
Epoch: 1800, time: 0.2s, loss: 2.181, acc:0.197 
Epoch: 2000, time: 0.3s, loss: 2.168, acc:0.204 
Epoch: 2200, time: 0.3s, loss: 2.156, acc:0.211 
Epoch: 2400, time: 0.2s, loss: 2.142, acc:0.218 
Epoch: 2600, time: 0.2s, loss: 2.128, acc:0.225 
Epoch: 2800, time: 0.3s, loss: 2.117, acc:0.231 
Epoch: 3000, time: 0.3s, loss: 2.103, acc:0.235 
Epoch: 3200, time: 0.2s, loss: 2.089, acc:0.242 
Epoch: 3400, time: 0.2s, loss: 2.075, acc:0.247 
Epoch: 3600, time: 0.3s, loss: 2.061, acc:0.254 
Epoch: 3800, time: 0.3s, loss: 2.047, acc:0.261 
Epoch: 4000, time: 0.2s, loss: 2.035, acc:0.268 
Epoch: 4200, time: 0.2s,

In [ ]:
model_RNN_80 = NeuralNetwork(nn.RNN, 80, embedding_size, num_hiddens, 80)
model, train_losses = train_model(model_RNN_80, data_80, n_epochs)

Epoch: 200, time: 0.6s, loss: 2.339, acc:0.114 
Epoch: 400, time: 0.6s, loss: 2.300, acc:0.121 
Epoch: 600, time: 0.5s, loss: 2.291, acc:0.129 
Epoch: 800, time: 0.6s, loss: 2.284, acc:0.132 
Epoch: 1000, time: 0.5s, loss: 2.279, acc:0.136 
Epoch: 1200, time: 0.6s, loss: 2.273, acc:0.141 
Epoch: 1400, time: 0.6s, loss: 2.268, acc:0.145 
Epoch: 1600, time: 0.6s, loss: 2.263, acc:0.149 
Epoch: 1800, time: 0.6s, loss: 2.257, acc:0.152 
Epoch: 2000, time: 0.6s, loss: 2.252, acc:0.155 
Epoch: 2200, time: 0.6s, loss: 2.249, acc:0.158 
Epoch: 2400, time: 0.6s, loss: 2.243, acc:0.161 
Epoch: 2600, time: 0.6s, loss: 2.239, acc:0.164 
Epoch: 2800, time: 0.5s, loss: 2.235, acc:0.167 
Epoch: 3000, time: 0.5s, loss: 2.231, acc:0.170 
Epoch: 3200, time: 0.6s, loss: 2.226, acc:0.173 
Epoch: 3400, time: 0.5s, loss: 2.223, acc:0.176 
Epoch: 3600, time: 0.6s, loss: 2.218, acc:0.179 
Epoch: 3800, time: 0.6s, loss: 2.215, acc:0.181 
Epoch: 4000, time: 0.6s, loss: 2.210, acc:0.185 
Epoch: 4200, time: 0.6s,

In [ ]:
model_GRU_10 = NeuralNetwork(nn.GRU, 10, embedding_size, num_hiddens, 10)
model, train_losses = train_model(model_GRU_10, data_10, n_epochs)

Epoch: 200, time: 0.2s, loss: 2.288, acc:0.128 
Epoch: 400, time: 0.2s, loss: 2.279, acc:0.136 
Epoch: 600, time: 0.2s, loss: 2.263, acc:0.146 
Epoch: 800, time: 0.2s, loss: 2.231, acc:0.161 
Epoch: 1000, time: 0.2s, loss: 2.171, acc:0.187 
Epoch: 1200, time: 0.2s, loss: 2.018, acc:0.260 
Epoch: 1400, time: 0.2s, loss: 1.930, acc:0.295 
Epoch: 1600, time: 0.2s, loss: 1.850, acc:0.325 
Epoch: 1800, time: 0.2s, loss: 1.758, acc:0.369 
Epoch: 2000, time: 0.2s, loss: 1.655, acc:0.409 
Epoch: 2200, time: 0.2s, loss: 1.542, acc:0.454 
Epoch: 2400, time: 0.2s, loss: 1.419, acc:0.507 
Epoch: 2600, time: 0.2s, loss: 1.284, acc:0.562 
Epoch: 2800, time: 0.2s, loss: 1.143, acc:0.615 
Epoch: 3000, time: 0.2s, loss: 0.992, acc:0.683 
Epoch: 3200, time: 0.2s, loss: 0.839, acc:0.740 
Epoch: 3400, time: 0.2s, loss: 0.698, acc:0.800 
Epoch: 3600, time: 0.2s, loss: 0.571, acc:0.846 
Epoch: 3800, time: 0.2s, loss: 0.467, acc:0.886 
Epoch: 4000, time: 0.2s, loss: 0.383, acc:0.914 
Epoch: 4200, time: 0.2s,

In [ ]:
model_GRU_40 = NeuralNetwork(nn.GRU, 40, embedding_size, num_hiddens, 40)
model, train_losses = train_model(model_GRU_40, data_40, n_epochs)

Epoch: 200, time: 0.6s, loss: 2.407, acc:0.105 
Epoch: 400, time: 0.6s, loss: 2.349, acc:0.112 
Epoch: 600, time: 0.6s, loss: 2.318, acc:0.118 
Epoch: 800, time: 0.6s, loss: 2.302, acc:0.117 
Epoch: 1000, time: 0.6s, loss: 2.294, acc:0.124 
Epoch: 1200, time: 0.6s, loss: 2.287, acc:0.128 
Epoch: 1400, time: 0.6s, loss: 2.281, acc:0.134 
Epoch: 1600, time: 0.6s, loss: 2.276, acc:0.139 
Epoch: 1800, time: 0.6s, loss: 2.270, acc:0.143 
Epoch: 2000, time: 0.6s, loss: 2.263, acc:0.146 
Epoch: 2200, time: 0.6s, loss: 2.256, acc:0.150 
Epoch: 2400, time: 0.6s, loss: 2.250, acc:0.155 
Epoch: 2600, time: 0.6s, loss: 2.242, acc:0.162 
Epoch: 2800, time: 0.6s, loss: 2.233, acc:0.166 
Epoch: 3000, time: 0.6s, loss: 2.224, acc:0.172 
Epoch: 3200, time: 0.6s, loss: 2.215, acc:0.178 
Epoch: 3400, time: 0.6s, loss: 2.206, acc:0.182 
Epoch: 3600, time: 0.6s, loss: 2.197, acc:0.188 
Epoch: 3800, time: 0.6s, loss: 2.187, acc:0.193 
Epoch: 4000, time: 0.6s, loss: 2.178, acc:0.197 
Epoch: 4200, time: 0.6s,

In [ ]:
model_GRU_80 = NeuralNetwork(nn.GRU, 80, embedding_size, num_hiddens, 80)
model, train_losses = train_model(model_GRU_80, data_80, n_epochs)

Epoch: 200, time: 1.3s, loss: 2.420, acc:0.103 
Epoch: 400, time: 1.3s, loss: 2.365, acc:0.105 
Epoch: 600, time: 1.3s, loss: 2.331, acc:0.110 
Epoch: 800, time: 1.3s, loss: 2.313, acc:0.114 
Epoch: 1000, time: 1.3s, loss: 2.305, acc:0.114 
Epoch: 1200, time: 1.4s, loss: 2.300, acc:0.119 
Epoch: 1400, time: 1.3s, loss: 2.296, acc:0.122 
Epoch: 1600, time: 1.3s, loss: 2.294, acc:0.124 
Epoch: 1800, time: 1.3s, loss: 2.292, acc:0.127 
Epoch: 2000, time: 1.3s, loss: 2.289, acc:0.130 
Epoch: 2200, time: 1.3s, loss: 2.287, acc:0.131 
Epoch: 2400, time: 1.3s, loss: 2.285, acc:0.132 
Epoch: 2600, time: 1.3s, loss: 2.283, acc:0.134 
Epoch: 2800, time: 1.3s, loss: 2.281, acc:0.136 
Epoch: 3000, time: 1.3s, loss: 2.278, acc:0.137 
Epoch: 3200, time: 1.3s, loss: 2.276, acc:0.139 
Epoch: 3400, time: 1.3s, loss: 2.274, acc:0.140 
Epoch: 3600, time: 1.3s, loss: 2.271, acc:0.142 
Epoch: 3800, time: 1.3s, loss: 2.269, acc:0.145 
Epoch: 4000, time: 1.3s, loss: 2.266, acc:0.147 
Epoch: 4200, time: 1.3s,

In [ ]:
model_LSTM_10 = NeuralNetwork(nn.LSTM, 10, embedding_size, num_hiddens, 10)
model, train_losses = train_model(model_LSTM_10, data_10, n_epochs)

Epoch: 200, time: 0.2s, loss: 2.292, acc:0.124 
Epoch: 400, time: 0.2s, loss: 2.288, acc:0.126 
Epoch: 600, time: 0.2s, loss: 2.282, acc:0.132 
Epoch: 800, time: 0.2s, loss: 2.272, acc:0.143 
Epoch: 1000, time: 0.2s, loss: 2.254, acc:0.153 
Epoch: 1200, time: 0.2s, loss: 2.221, acc:0.168 
Epoch: 1400, time: 0.2s, loss: 2.151, acc:0.198 
Epoch: 1600, time: 0.2s, loss: 2.035, acc:0.252 
Epoch: 1800, time: 0.2s, loss: 1.918, acc:0.304 
Epoch: 2000, time: 0.2s, loss: 1.810, acc:0.347 
Epoch: 2200, time: 0.2s, loss: 1.702, acc:0.394 
Epoch: 2400, time: 0.2s, loss: 1.576, acc:0.446 
Epoch: 2600, time: 0.2s, loss: 1.448, acc:0.497 
Epoch: 2800, time: 0.2s, loss: 1.301, acc:0.558 
Epoch: 3000, time: 0.2s, loss: 1.170, acc:0.611 
Epoch: 3200, time: 0.2s, loss: 1.029, acc:0.673 
Epoch: 3400, time: 0.2s, loss: 0.900, acc:0.727 
Epoch: 3600, time: 0.2s, loss: 0.774, acc:0.775 
Epoch: 3800, time: 0.2s, loss: 0.692, acc:0.803 
Epoch: 4000, time: 0.2s, loss: 0.671, acc:0.798 
Epoch: 4200, time: 0.2s,

In [ ]:
model_LSTM_40 = NeuralNetwork(nn.LSTM, 40, embedding_size, num_hiddens, 40)
model, train_losses = train_model(model_LSTM_40, data_40, n_epochs)

Epoch: 200, time: 0.8s, loss: 2.398, acc:0.104 
Epoch: 400, time: 0.8s, loss: 2.337, acc:0.115 
Epoch: 600, time: 0.8s, loss: 2.314, acc:0.117 
Epoch: 800, time: 0.8s, loss: 2.307, acc:0.121 
Epoch: 1000, time: 0.8s, loss: 2.297, acc:0.122 
Epoch: 1200, time: 0.8s, loss: 2.292, acc:0.127 
Epoch: 1400, time: 0.8s, loss: 2.285, acc:0.132 
Epoch: 1600, time: 0.8s, loss: 2.280, acc:0.136 
Epoch: 1800, time: 0.8s, loss: 2.273, acc:0.142 
Epoch: 2000, time: 0.8s, loss: 2.266, acc:0.147 
Epoch: 2200, time: 0.8s, loss: 2.259, acc:0.152 
Epoch: 2400, time: 0.8s, loss: 2.248, acc:0.160 
Epoch: 2600, time: 0.8s, loss: 2.239, acc:0.164 
Epoch: 2800, time: 0.8s, loss: 2.230, acc:0.168 
Epoch: 3000, time: 0.8s, loss: 2.219, acc:0.173 
Epoch: 3200, time: 0.8s, loss: 2.206, acc:0.182 
Epoch: 3400, time: 0.8s, loss: 2.195, acc:0.188 
Epoch: 3600, time: 0.8s, loss: 2.185, acc:0.195 
Epoch: 3800, time: 0.8s, loss: 2.172, acc:0.200 
Epoch: 4000, time: 0.8s, loss: 2.161, acc:0.205 
Epoch: 4200, time: 0.8s,

In [ ]:
model_LSTM_80 = NeuralNetwork(nn.LSTM, 80, embedding_size, num_hiddens, 80)
model, train_losses = train_model(model_LSTM_80, data_80, n_epochs)

Epoch: 200, time: 1.5s, loss: 2.451, acc:0.102 
Epoch: 400, time: 1.5s, loss: 2.369, acc:0.104 
Epoch: 600, time: 1.5s, loss: 2.342, acc:0.109 
Epoch: 800, time: 1.5s, loss: 2.323, acc:0.113 
Epoch: 1000, time: 1.5s, loss: 2.315, acc:0.114 
Epoch: 1200, time: 1.5s, loss: 2.308, acc:0.115 
Epoch: 1400, time: 1.5s, loss: 2.304, acc:0.116 
Epoch: 1600, time: 1.5s, loss: 2.300, acc:0.118 
Epoch: 1800, time: 1.5s, loss: 2.297, acc:0.121 
Epoch: 2000, time: 1.5s, loss: 2.295, acc:0.125 
Epoch: 2200, time: 1.5s, loss: 2.292, acc:0.126 
Epoch: 2400, time: 1.5s, loss: 2.290, acc:0.128 
Epoch: 2600, time: 1.5s, loss: 2.288, acc:0.130 
Epoch: 2800, time: 1.5s, loss: 2.286, acc:0.132 
Epoch: 3000, time: 1.5s, loss: 2.283, acc:0.134 
Epoch: 3200, time: 1.5s, loss: 2.281, acc:0.136 
Epoch: 3400, time: 1.5s, loss: 2.278, acc:0.139 
Epoch: 3600, time: 1.5s, loss: 2.276, acc:0.141 
Epoch: 3800, time: 1.5s, loss: 2.273, acc:0.143 
Epoch: 4000, time: 1.5s, loss: 2.270, acc:0.146 
Epoch: 4200, time: 1.5s,

|net| len str|time, s |loss	|acc|
|:------|:------|------:|------:|------:|
|RNN|10|0.1|0.161|0.942|
||40|0.2|1.963|0.299|
||80|0.6|2.190|0.196|
|GRU|10|0.2|0.192|0.945|
||40|0.6|2.129|0.221|
||80|1.3|2.252|0.157|
|LSTM|10|0.2|0.312|0.922|
||40|0.8|2.095|0.240|
||80|1.5|2.253|0.157|


На последовательностях длиной 10 RNN, GRU справились почти одинаково. С увеличением длины последовательности увеличивается время обучения, loss и accuracy. На последовательностях длиной 40 и 80 сети с GRU и LSTM показывают почти одинаковые результаты, правда LSTM чуть медленнее. Но в данном примере на длинных последовательностях RNN оказался чуть лучше, чем GRU и LSTM.

#Задание 2:
применить LSTM для решения лекционного практического задания

In [ ]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2023-02-03 12:42:05--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.204.216, 52.217.161.184, 54.231.202.176, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.204.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  1.86MB/s    in 0.3s    

2023-02-03 12:42:05 (1.86 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [ ]:
import re

In [ ]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [ ]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [ ]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [ ]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]), tensor(23))

In [ ]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions.squeeze(0)

In [ ]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [ ]:
model(X[0:1])

tensor([[-0.0226,  0.0160, -0.0708, -0.0347,  0.0859,  0.1034,  0.0247,  0.0741,
          0.1761, -0.0089,  0.0785,  0.1652, -0.0007,  0.0308,  0.0326, -0.0449,
          0.0804, -0.0630, -0.0349, -0.1179,  0.0869, -0.1890, -0.0987, -0.0741,
          0.0055,  0.1189, -0.1021]], grad_fn=<SqueezeBackward1>)

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred)[0]
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [ ]:
generate_text()

ds inserted here the old theological pro|qqpcly klkqlnrihayk ahmrphzykllga neymxj


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for ep in range(5):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b, y_b
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

<ipython-input-30-8f474e72f798>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for X_b, y_b in tqdm.tqdm_notebook(data):


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 0. Time: 126.470, Train loss: 2.164
l qualities that it is a synthesis of al| whord fol jalat theas and noce dejury r


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 1. Time: 127.029, Train loss: 1.779
the same language or rather when people |wherther eve ermirit itsis realite theer


...

Epoch 2. Time: 125.846, Train loss: 1.637
ld even say craftiness with which the pr|amptious on the femment of t the greates


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 3. Time: 127.402, Train loss: 1.550
o any other reality but just that of our| case s pusty what he beyther they exist


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 4. Time: 127.212, Train loss: 1.491
qual suddenness transforms into penitent| abest thereffully thic self president f
